In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install tf-nightly==2.4.0.dev20200929

     |████████████████████████████████| 391.7MB 34kB/s 
     |████████████████████████████████| 471kB 37.1MB/s 
     |████████████████████████████████| 10.6MB 43.4MB/s 


In [1]:
!gsutil -q -m cp -r /content/drive/My\ Drive/SocialDistance/DPDMEfficientDetD0/fine_tuned_model /content/drive/My\ Drive/SocialDistance/DPDM_D0_tflite/

In [3]:
import tensorflow as tf
tf.__version__

'2.4.0-dev20200929'

In [5]:
%cd /content/drive/My\ Drive/SocialDistance/DPDM_D0_tflite/
%ls 'fine_tuned_model/saved_model'
saved_model_dir = 'fine_tuned_model/saved_model'


model = tf.saved_model.load(saved_model_dir)
model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY].inputs[0].set_shape([1, 512, 512, 3])
tf.saved_model.save(model, "saved_model_new", signatures=model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY])

/content/drive/My Drive/SocialDistance/DPDM_D0_tflite
saved_model.pb  variables/
INFO:tensorflow:Assets written to: saved_model_new/assets


In [6]:
# Convert
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir='saved_model_new', signature_keys=['serving_default'])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()

In [8]:
import numpy as np


## TFLite Interpreter to check input shape
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Print input and output tensors
print("INPUT DETAILS: \n",input_details)
print("OUTPUT DETAILS: \n",output_details)
# Test the model on random input data.
input_shape = input_details[0]['shape']
print(input_shape)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

INPUT DETAILS: 
 [{'name': 'serving_default_input_tensor:0', 'index': 0, 'shape': array([  1, 512, 512,   3], dtype=int32), 'shape_signature': array([  1, 512, 512,   3], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
OUTPUT DETAILS: 
 [{'name': 'StatefulPartitionedCall:4', 'index': 66519, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([ 1, -1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'StatefulPartitionedCall:6', 'index': 66380, 'shape': array([    1, 49104,     4], dtype=int32), 'shape_signature': array([    1, 49104,     4], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0

In [7]:
#tflite_file = '/content/fine_tuned_model/mask_detector.tflite'
tflite_file = '/content/drive/My Drive/SocialDistance/DPDM_D0_tflite/fine_tuned_model/DPDMD0_person_detector.tflite'
#%cd /drive/My\ Drive/Maithri_TF/fine_tuned_model/
%cd /content/drive/My\ Drive/SocialDistance/DPDMEfficientDetD0/fine_tuned_model/
#now save the tflite model to the file
#tflite_model.save(tflite_file)   #Note this does not seem to work although in google documentation
open(tflite_file, "wb").write(tflite_model)
!ls /content/drive/My\ Drive/SocialDistance/DPDM_D0_tflite/fine_tuned_model

/content/drive/My Drive/SocialDistance/DPDMEfficientDetD0/fine_tuned_model
checkpoint		       pipeline.config	valid_pipeline.config
DPDMD0_person_detector.tflite  saved_model


In [8]:
from google.colab import files
#files.download(tflite_model)

!ls /content/drive/My\ Drive/SocialDistance/DPDM_D0_tflite/fine_tuned_model

files.download('/content/drive/My Drive/SocialDistance/DPDM_D0_tflite/fine_tuned_model/DPDMD0_person_detector.tflite')

checkpoint		       pipeline.config	valid_pipeline.config
DPDMD0_person_detector.tflite  saved_model


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
import numpy as np
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="/content/drive/My Drive/SocialDistance/DPDM_D0_tflite/fine_tuned_model/DPDMD0_person_detector.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


### LOADING IMAGE
import cv2
def load_rgb_image_into_numpy_array(path):
  # read in image as grayscale cv2 image
  img = cv2.imread(path, cv2.IMREAD_COLOR)
  im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return im_rgb


# download test image
test_image = load_rgb_image_into_numpy_array('/content/drive/My Drive/SocialDistance/DPDMEfficientDetD0/Data/TestData_DPDM/images/2.jpg')

# RESIZE TO 512 x 512

# set a new height in pixels
new_height = 512
new_width = 512
dsize = (new_width, new_height)
# resize image
scaled_image = cv2.resize(test_image, dsize, interpolation = cv2.INTER_AREA)
# generate tensor
input_tensor = tf.convert_to_tensor(np.expand_dims(scaled_image, 0), dtype=tf.uint8)

print(input_tensor.shape)

interpreter.set_tensor(input_details[0]['index'], input_tensor)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)
print("output array length: " + str(len(output_data[0])))

(1, 512, 512, 3)
[[0.9957597  0.9684224  0.880913   0.69075537 0.6644237  0.47980133
  0.46004784 0.06493288 0.05839229 0.0530003  0.05154696 0.05086136
  0.04633224 0.04167682 0.0415034  0.04107836 0.03659123 0.03500023
  0.0322955  0.03004518 0.02906421 0.02866599 0.02715203 0.02612799
  0.02337852 0.02259013 0.02200839 0.02154994 0.01997665 0.0195356
  0.01800448 0.01799226 0.01749739 0.01710692 0.01688513 0.01651755
  0.01616323 0.0160442  0.01574534 0.01570481 0.01569521 0.01532039
  0.01447526 0.01380828 0.01369467 0.01332656 0.01314074 0.01312479
  0.01274678 0.01270795 0.01268455 0.01225951 0.01199609 0.01183662
  0.01147765 0.01141015 0.01126695 0.01116422 0.01107255 0.01093975
  0.01088384 0.01073369 0.01064429 0.01043907 0.01040789 0.01034102
  0.01015553 0.00999424 0.00996715 0.00991848 0.00963137 0.00961205
  0.00954965 0.00940585 0.00926062 0.00910234 0.00909692 0.00899801
  0.00874048 0.0087342  0.00872663 0.00859123 0.00853094 0.00852528
  0.00842687 0.00833461 0.008187